In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
#categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']

In [3]:
#fetching train data
twenty_train = fetch_20newsgroups(subset='train',shuffle=True, random_state=42)
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [4]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))
print(twenty_train.target_names[twenty_train.target[0]])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
rec.autos


In [5]:
parameters = {'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3),'vect__ngram_range': [(1, 1), (1, 2)]}

In [6]:
count_vect = CountVectorizer()
#gs_X_train_counts = GridSearchCV(count_vect.fit_transform(twenty_train.data),parameter_bog,n_jobs=-1)
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 130107)

In [7]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

In [8]:
#Fetching test data
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)
X_test_counts = count_vect.transform(twenty_test.data)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [9]:
# Naive Bayes Classifier
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
predicted = clf.predict(X_test_tfidf)
print np.mean(predicted == twenty_test.target)
#for doc, category in zip(docs_new, predicted):
#    print('%r => %s' % (doc, twenty_train.target_names[category]))

0.77389803505


In [10]:
# SVM Classifier
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42).fit(X_train_tfidf, twenty_train.target)
predicted = clf.predict(X_test_tfidf)
print np.mean(predicted == twenty_test.target)

0.823818374934


In [ ]:
#pipeline with NB classifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
text_clf = text_clf.fit(twenty_train.data,twenty_train.target)
predicted = text_clf.predict(twenty_test.data)
print np.mean(predicted == twenty_test.target)

In [27]:
# pipeline with SVM classifier
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf',SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42) )])
text_clf = text_clf.fit(twenty_train.data,twenty_train.target)
predicted = text_clf.predict(twenty_test.data)
print np.mean(predicted == twenty_test.target)

0.823818374934


In [30]:
#pipeliine with gridsearchcv
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf',SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42) )])
gs_clf = GridSearchCV(text_clf,parameters,n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data,twenty_train.target)
predicted = gs_clf.predict(twenty_test.data)
print np.mean(predicted == twenty_test.target)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.